In [1]:
# numpy
import numpy as np
from numpy import unravel_index
print( 'Using numpy version %s' % np.__version__ )

# system
import glob, os

# astropy
import astropy as astro
from astropy.table import Table
from astropy.io import ascii, fits
from astropy.stats import sigma_clip
from astropy.convolution import convolve_fft
import astropy.units as u
from astropy.cosmology import Planck15
from astropy.coordinates import SkyCoord
from astropy.nddata.utils import Cutout2D
from astropy.wcs import WCS
print( 'Using astropy version %s' % astro.__version__ )

Using numpy version 1.21.5
Using astropy version 4.3.1


### Load catalog of sources

In [5]:
cat_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/final_sample_id_list.txt')
#cat_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/sample_lists/remaining_non_optimal_galaxies.txt')
print( len( cat_f ), cat_f.colnames )

fields = ['goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodsn', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss', 'goodss']
ids = [10458, 10832, 11879, 12330, 13233, 16197, 17050, 17983, 19591, 20052, 20093, 20367, 21006, 23127, 23375, 23485, 24061, 26799, 26883, 2744, 3772, 3891, 4013, 4651, 4911, 5758, 5974, 602, 6298, 6446, 6545, 8771, 9223, 10303, 12028, 12488, 12779, 14306, 1446, 16264, 19462, 20172, 20310, 23246, 24990, 2551, 25716, 3503, 4210, 5223, 6423, 7365, 8352]
zs = [0.9917, 0.4748, 1.489, 0.688, 0.436, 0.409, 1.312, 1.331, 0.8541, 1.372, 0.849, 0.484, 0.8355, 0.9048, 1.0306, 0.6341, 1.236, 0.7465, 1.435, 0.4097, 0.8403, 0.7504, 1.0148, 0.6393, 0.578, 1.3512, 0.6494, 1.427, 1.052, 0.996, 1.3229, 0.4336, 0.4538, 0.9606, 0.732, 0.629, 0.678, 0.731, 1.298, 0.765, 1.166, 0.525, 0.605, 1.2458, 1.174, 1.4181, 1.2333, 1.26, 0.7396, 1.377, 0.991, 0.7341, 0.715]

sample_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/sed_fitting_catalogs/goods_sample_param_z_dependent_size_cut_good_photo_z_ccww_dust_law.txt')


1703 ['ID', 'field', 'z', 'RA', 'DEC', 'log_Mstar', 'log_SFR', 'PA_SA', 'axis_ratio_SA', 'axis_ratio_SE', 'PA_SE', 'axis_ratio_VDW', 'PA_VDW', 'Reff_VDW', 'n_VDW', 'flag_VDW', 'UV', 'VJ']


In [13]:
cat_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/sed_fitting_catalogs/goods_sample_param_z_dependent_size_cut_good_photo_z_all_assumptions_visual_classifications.txt')

img_crit = (cat_f['category'] >= 1)

print(len(cat_f[img_crit]))

fields = np.load('/Users/vega/CANDELS/sed_assumptions_paper/median_profiles_test/low_z_low_mass_qg_profile_fields.npy')
gal_ids = np.load('/Users/vega/CANDELS/sed_assumptions_paper/median_profiles_test/low_z_low_mass_qg_profile_ids.npy')


1759


### goodss catalog

In [3]:
ids = [45, 1772, 2464, 4410, 5626, 8513, 8541, 9242, 9405, 10348, 11981, 12948, 16135, 16929, 18227, 21341, 21401, 21416, 21919, 22063, 22303, 22386, 22814, 23835, 24172, 24629, 24855]

# leftover galaxies from the intermediate tau gauss fits (fourth set of assumptions) - Nov 26, 2023

ids = [17482]

cat_f = ascii.read('/Volumes/Vega/CANDELS/sed_assumptions_paper/goodss_candels_kodra_luo_vdw_hb_photo_z_cat.txt')

### goodsn catalog

In [7]:
cat_f = ascii.read('/Volumes/Vega/CANDELS/sed_assumptions_paper/goodsn_candels_kodra_xue_vdw_hb_photo_z_cat.txt')

ids = [7479, 10940, 11884, 15649, 15997, 16350, 16915, 17396, 18225, 21643, 22579]

# leftover galaxies from the intermediate tau gauss fits (fourth set of assumptions) - Nov 26, 2023

ids = [1080, 5304, 16680, 21258]


In [2]:
cat_f = ascii.read('/Users/vega/CANDELS/sed_assumptions_paper/goods_sample_remaining_final_cuts_no_vis.txt')

### make images

In [8]:
bands = ['435', '606', '775', '850', '125', '140', '160']
#bands = ['814']

length = 200

for i in range( len( cat_f ) ):
    
    gal_id = cat_f['ID'][i]
    #field = cat_f['field'][i] #fields[i]
    field = 'goodsn'
    
    if gal_id not in ids:
        continue
    
    #cat_ind = np.where( (cat_f['ID'] == gal_id) & (cat_f['field'] == field) )[0]
    
    #coords = SkyCoord(ra=cat_f['RA'][cat_ind][0], dec=cat_f['DEC'][cat_ind][0], unit='deg')
    coords = SkyCoord(cat_f['RA'][i], cat_f['DEC'][i], unit='deg')
    
    #cat_f['field'][img_crit][i] == 'goodss':
    
    if field == "goodss":
        
        for b in bands:
    
            band_f_list = glob.glob( '/Volumes/Vega/CANDELS_data/GOODSS/goodss_*%s*' % b )
    
            if b != '160':
                kernel_f = glob.glob('/Volumes/Vega/3DHST/GOODSS/goodss_*_psf/*%sW_kernel.fits.gz' % b)
                band_ext = 'w'
    
                if len( kernel_f ) == 0:
                    kernel_f = glob.glob('/Volumes/Vega/3DHST/GOODSS/goodss_*_psf/*%sWcand_kernel.fits.gz' % b)
        
                    if len( kernel_f ) == 0:
                        kernel_f = glob.glob('/Volumes/Vega/3DHST/GOODSS/goodss_*_psf/*%sLP_kernel.fits.gz' % b)
                        band_ext = 'lp'
                    
                k_f = kernel_f[0]
    
            drz_ind = np.where( np.array( ['drz' in f for f in band_f_list] ) )[0][0]
            exp_ind = np.where( np.array( ['exp' in f for f in band_f_list] ) )[0][0]
            wht_ind = np.where( np.array( ['wht' in f for f in band_f_list] ) )[0][0]
    
            # candels mosaic
            hdulist_c = fits.open(band_f_list[drz_ind])
            hdu_c = hdulist_c[0]
            wcs_c = WCS(hdu_c.header)
            wcs_c.sip = None

            # weight image
            hdulist_w = fits.open(band_f_list[wht_ind])
            hdu_w = hdulist_w[0]
            wcs_w = WCS(hdu_w.header)
            wcs_w.sip = None

            # exposure time image
            hdulist_exp = fits.open(band_f_list[exp_ind])
            hdu_exp = hdulist_exp[0]
            wcs_exp = WCS(hdu_exp.header)
            wcs_exp.sip = None

            # 3dhst psf-matching kernel
            hdulist_k = fits.open(k_f)
            hdu_k = hdulist_k[0]

            # create cutouts
            cutout_c = Cutout2D(hdu_c.data, coords, (length, length), wcs=wcs_c)
            cutout_w = Cutout2D(hdu_w.data, coords, (length, length), wcs=wcs_w)
            cutout_exp = Cutout2D(hdu_exp.data, coords, (length, length), wcs=wcs_exp)
    
            if b != '160':
                cutout_c_conv = convolve_fft( cutout_c.data, hdu_k.data)
                #cutout_w_conv = convolve_fft( cutout_w.data, hdu_k.data)
                cutout_w = cutout_w.data
                cutout_exp = cutout_exp.data
            else:
                cutout_c_conv = cutout_c.data
                cutout_w = cutout_w.data
                cutout_exp = cutout_exp.data

            img_wcs = cutout_c.wcs.to_header()
    
            if 'b' == '125' or b == '140' or b == '160':
                convert_factor = hdu_c.header['PHOTFNU']
            else:
                convert_factor = hdu_c.header['PHOTFLAM'] * hdu_c.header['PHOTPLAM'] ** 2 * 1e23 / (2.99792458e18)
    
            hdu1 = fits.HDUList()
            hdu1.append(fits.PrimaryHDU(header=img_wcs))
            hdu1.append(fits.ImageHDU())
            hdu1.append(fits.ImageHDU())
            hdu1.append(fits.ImageHDU())

            header = hdu1[0].header

            hdu1[0].data = cutout_c_conv
            hdu1[1].data = cutout_w
            hdu1[2].data = cutout_exp

            header['Ext1'] = ('Science image')
            header['Ext2'] = ('Weight image')
            header['Ext3'] = ('Exposure time image')
            header['PHOT_ZP'] = (convert_factor, 'conversion factor to measure flux in Jy')

            hdu1.writeto('/Volumes/Vega/CANDELS/candels_images/goodss_%d_f%s%s_images.fits' % (gal_id, b, band_ext), overwrite=True)
            
            hdulist_c.close()
            hdulist_exp.close()
            hdulist_k.close()
            hdulist_w.close()
            
    elif field == "goodsn":
        
        # the GOODS-N ACS mosaics are missing certain keywords from the headers that
        # prevent astropy from reading the WCS in the header
        # as a quick and easy solution, I'm copying the header from the F160W mosaic
        hdulist_160 = fits.open('/Volumes/Vega/CANDELS_data/GOODSN/goodsn_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
        hdu_160 = hdulist_160[0]
        wcs_160 = WCS(hdu_160.header)
        wcs_160.sip = None
        
        for b in bands:
    
            band_f_list = glob.glob( '/Volumes/Vega/CANDELS_data/GOODSN/goodsn_*%s*' % b )
    
            if b != '160':
                kernel_f = glob.glob('/Volumes/Vega/3DHST/GOODSN/goodsn_*_psf/*%sW_kernel.fits.gz' % b)
                band_ext = 'w'
    
                if len( kernel_f ) == 0:
                    if b == '814':
                        kernel_f = glob.glob('/Volumes/Vega/3DHST/GOODSS/goodss_3dhst_v4.0_acs_psf/goodss_3dhst.v4.0.F814Wcand_kernel.fits.gz')
                        band_ext = 'w'
        
                    elif b == '850':
                        kernel_f = glob.glob('/Volumes/Vega/3DHST/GOODSN/goodsn_*_psf/*%sLP_kernel.fits.gz' % b)
                        band_ext = 'lp'
                        
                k_f = kernel_f[0]
                
            drz_ind = np.where( np.array( ['drz' in f for f in band_f_list] ) )[0][0]
            exp_ind = np.where( np.array( ['exp' in f for f in band_f_list] ) )[0][0]
            wht_ind = np.where( np.array( ['wht' in f for f in band_f_list] ) )[0][0]
            
            # candels mosaic
            hdulist_c = fits.open(band_f_list[drz_ind])
            hdu_c = hdulist_c[0]
            
            # weight image
            hdulist_w = fits.open(band_f_list[wht_ind])
            hdu_w = hdulist_w[0]
            #wcs_w = WCS(hdu_w.header)
            #wcs_w.sip = None

            # exposure time image
            hdulist_exp = fits.open(band_f_list[exp_ind])
            hdu_exp = hdulist_exp[0]
            #wcs_exp = WCS(hdu_exp.header)
            #wcs_exp.sip = None

            # 3dhst psf-matching kernel
            hdulist_k = fits.open(k_f)
            hdu_k = hdulist_k[0]

            # create cutouts
            cutout_c = Cutout2D(hdu_c.data, coords, (length, length), wcs=wcs_160)
            cutout_w = Cutout2D(hdu_w.data, coords, (length, length), wcs=wcs_160)
            cutout_exp = Cutout2D(hdu_exp.data, coords, (length, length), wcs=wcs_160)
    
            if b != '160':
                cutout_c_conv = convolve_fft( cutout_c.data, hdu_k.data)
                cutout_w_conv = convolve_fft( cutout_w.data, hdu_k.data)
                cutout_exp = cutout_exp.data
            else:
                cutout_c_conv = cutout_c.data
                cutout_w_conv = cutout_w.data
                cutout_exp = cutout_exp.data

            img_wcs = cutout_c.wcs.to_header()
    
            if 'b' == '125' or b == '140' or b == '160':
                convert_factor = hdu_c.header['PHOTFNU']
            else:
                convert_factor = hdu_c.header['PHOTFLAM'] * hdu_c.header['PHOTPLAM'] ** 2 * 1e23 / (2.99792458e18)
    
            hdu1 = fits.HDUList()
            hdu1.append(fits.PrimaryHDU(header=img_wcs))
            hdu1.append(fits.ImageHDU())
            hdu1.append(fits.ImageHDU())
            hdu1.append(fits.ImageHDU())

            header = hdu1[0].header

            hdu1[0].data = cutout_c_conv
            hdu1[1].data = cutout_w_conv
            hdu1[2].data = cutout_exp

            header['Ext1'] = ('Science image')
            header['Ext2'] = ('Weight image')
            header['Ext3'] = ('Exposure time image')
            header['PHOT_ZP'] = (convert_factor, 'conversion factor to measure flux in Jy')

            hdu1.writeto('/Volumes/Vega/CANDELS/candels_images/goodsn_%d_f%s%s_images.fits' % (gal_id, b, band_ext), overwrite=True)
            
            hdulist_160.close()
            hdulist_c.close()
            hdulist_exp.close()
            hdulist_k.close()
            hdulist_w.close()


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


### Create 3D-HST images

In [6]:
bands = ['435', '606', '775', '850', '125', '140', '160']

length = 200

for i in [1112]: #range( len( cat_f ) ):
    
    gal_id = cat_f['ID'][i]
    
    coords = SkyCoord(ra=cat_f['RA'][i], dec=cat_f['DEC'][i], unit='deg')
    
    if cat_f['field'][i] == 'goodss':
        
        for b in bands:
    
            band_f_list = glob.glob( '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f%s*/*.fits' % b)
            
            band_ext = 'w'
    
            if b != '160':
                sci_ind = np.where( np.array( ['conv_sci' in f for f in band_f_list] ) )[0][0]
                
                if b == '850':
                    band_ext = 'lp'
                
            else:
                sci_ind = np.where( np.array( ['sci' in f for f in band_f_list] ) )[0][0]
                
            print( band_f_list )
                
            wht_ind = np.where( np.array( ['wht' in f for f in band_f_list] ) )[0][0]
            #exp_ind = np.where( np.array( ['_exp' in f for f in band_f_list] ) )[0][0]
    
            # candels mosaic
            hdulist_c = fits.open(band_f_list[sci_ind])
            hdu_c = hdulist_c[0]
            wcs_c = WCS(hdu_c.header)
            wcs_c.sip = None

            # weight image
            hdulist_w = fits.open(band_f_list[wht_ind])
            hdu_w = hdulist_w[0]
            wcs_w = WCS(hdu_w.header)
            wcs_w.sip = None
            
            # exposure time image
            #hdulist_exp = fits.open(band_f_list[exp_ind])
            #hdu_exp = hdulist_exp[0]
            #wcs_exp = WCS(hdu_exp.header)
            #wcs_exp.sip = None

            # create cutouts
            cutout_c = Cutout2D(hdu_c.data, coords, (length, length), wcs=wcs_c)
            cutout_w = Cutout2D(hdu_w.data, coords, (length, length), wcs=wcs_w)
            #cutout_exp = Cutout2D(hdu_exp.data, coords, (length, length), wcs=wcs_exp)
    
            cutout_c_conv = cutout_c.data
            cutout_w = cutout_w.data
            #cutout_exp = cutout_exp.data

            img_wcs = cutout_c.wcs.to_header()
    
            if 'b' == '125' or b == '140' or b == '160':
                convert_factor = hdu_c.header['PHOTFNU']
            else:
                convert_factor = hdu_c.header['PHOTFLAM'] * hdu_c.header['PHOTPLAM'] ** 2 * 1e23 / (2.99792458e18)
    
            hdu1 = fits.HDUList()
            hdu1.append(fits.PrimaryHDU(header=img_wcs))
            hdu1.append(fits.ImageHDU())
            hdu1.append(fits.ImageHDU())
            #hdu1.append(fits.ImageHDU())

            header = hdu1[0].header

            hdu1[0].data = cutout_c_conv
            hdu1[1].data = cutout_w
            #hdu1[2].data = cutout_exp

            header['Ext1'] = ('Science image')
            header['Ext2'] = ('Weight image')
            #header['Ext3'] = ('Exposure time image')
            header['PHOT_ZP'] = (convert_factor, 'conversion factor to measure flux in Jy')

            hdu1.writeto('/Volumes/Kassin/CANDELS/3dhst_images/goodss_%d_f%s%s_images.fits' % (gal_id, b, band_ext), overwrite=True)
            
            hdulist_c.close()
            hdulist_w.close()



['/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f435w/goodss_3dhst.v4.0.F435W_conv_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f435w/goodss_3dhst.v4.0.F435W_orig_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f435w/goodss_3dhst.v4.0.F435W_orig_wht.fits']


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


['/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f606w/goodss_3dhst.v4.0.F606W_conv_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f606w/goodss_3dhst.v4.0.F606W_orig_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f606w/goodss_3dhst.v4.0.F606W_orig_wht.fits']
['/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f775w/goodss_3dhst.v4.0.F775W_conv_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f775w/goodss_3dhst.v4.0.F775W_orig_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f775w/goodss_3dhst.v4.0.F775W_orig_wht.fits']
['/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f850lp/goodss_3dhst.v4.0.F850LP_conv_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f850lp/goodss_3dhst.v4.0.F850LP_orig_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f850lp/goodss_3dhst.v4.0.F850LP_orig_wht.fits']
['/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f125w/goodss_3dhst.v4.0.F125W_conv_sci.fits', '/Volumes/Kassin/3DHST/GOODSS/goodss_3dhst_v4.0_f1

### Create segmentation map images

In [7]:
# segmap
hdulist_segmap_gds = fits.open('/Volumes/Kassin/CANDELS_data/CANDELS-Catalog-Team-Release-May-2015/GOODSS/gs_max_sx_h_130209_hphotom_comb_seg_psfmatch2h_p01.fits')
hdu_segmap_gds = hdulist_segmap_gds[0]
wcs_segmap_gds = WCS(hdu_segmap_gds.header)
wcs_segmap_gds.sip = None

hdulist_segmap_gdn = fits.open('/Volumes/Kassin/CANDELS_data/CANDELS-Catalog-Team-Release-May-2015/GOODSN/gn_all_candels_sx_h_130829_hphotom_comb_seg_psfmatch2h.fits')
hdu_segmap_gdn = hdulist_segmap_gdn[0]
wcs_segmap_gdn = WCS(hdu_segmap_gdn.header)
wcs_segmap_gdn.sip = None

length = 200
c = int( 0.5 * (length - 1) )

for i in range( len( cat_f ) ):
    
    gal_id = cat_f['ID'][i]
    
    coords = SkyCoord(ra=cat_f['RA'][i], dec=cat_f['DEC'][i], unit='deg')
    
    if cat_f['field'][i] == 'goodss':
        
        cutout_segmap = Cutout2D(hdu_segmap_gds.data, coords, (length, length), wcs=wcs_segmap_gds)
        
        # change pixel values corresponding to galaxies other than central galaxy to -1
        segmap = np.where( (cutout_segmap.data > 0) & (cutout_segmap.data != cutout_segmap.data[c,c]), -1, cutout_segmap.data) 
        
        # turn other galaxy pixels to zero
        segmap = np.ones( (length, length) ) + segmap
        
        # change pixels of central galaxy to 1
        segmap = np.where( cutout_segmap.data == cutout_segmap.data[c,c], 1, segmap) 
        
        np.save( '/Volumes/Kassin/CANDELS/candels_images/goodss_%d_segmap.npy' % (gal_id), segmap )
        
    elif cat_f['field'][i] == 'goodsn':
        
        cutout_segmap = Cutout2D(hdu_segmap_gdn.data, coords, (length, length), wcs=wcs_segmap_gdn)
        
        # change pixel values corresponding to galaxies other than central galaxy to -1
        segmap = np.where( (cutout_segmap.data > 0) & (cutout_segmap.data != cutout_segmap.data[c,c]), -1, cutout_segmap.data) 
        
        # turn other galaxy pixels to zero
        segmap = np.ones( (length, length) ) + segmap
        
        # change pixels of central galaxy to 1
        segmap = np.where( cutout_segmap.data == cutout_segmap.data[c,c], 1, segmap) 
        
        np.save( '/Volumes/Kassin/CANDELS/candels_images/goodsn_%d_segmap.npy' % (gal_id), segmap )
        

hdulist_segmap_gds.close()
hdulist_segmap_gdn.close()


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
INFO: 
             

### Convolving mosaics for sky subtraction error calculations

In [9]:
# use 981 as the index of the ~center of GOODS-S
# use 189 as the index of the ~center of GOODS-N

i = 981

gal_id = cat_f['ID'][i]
field = cat_f['field'][i]

coords = SkyCoord(ra=cat_f['RA'][i], dec=cat_f['DEC'][i], unit='deg')

# trying to convolve a 15000 y x 15000 x pixel cutout works!! (good size for ~95% of GOODS-N)
# trying to convolve a 16000 y x 13000 x pixel cutout works!! (good size for ~98% of GOODS-S)

if field == 'goodsn':
    length_y, length_x = 15000, 15000
    
    hdulist_160 = fits.open('/Volumes/BEAGLE1/CANDELS_data/GOODSN/goodsn_all_wfc3_ir_f160w_060mas_v1.0_drz.fits')
    hdu_160 = hdulist_160[0]
    wcs_c = WCS(hdu_160.header)
    
elif field == 'goodss':
    length_y, length_x = 16000, 13000

b = '435w'
    
# candels mosaic
hdulist_c = fits.open('/Volumes/BEAGLE1/CANDELS_data/GOODSS/goodss_all_acs_wfc_f435w_060mas_v1.5_drz.fits')
hdu_c = hdulist_c[0]
if field == 'goodss':
    wcs_c = WCS(hdu_c.header)
        
# psf matching kernel
hdulist_k = fits.open('/Volumes/BEAGLE1/3DHST/GOODSS/goodss_3dhst_v4.0_acs_psf/goodss_3dhst.v4.0.F435W_kernel.fits.gz')
hdu_k = hdulist_k[0]

cutout_c = Cutout2D(hdu_c.data, coords, (length_y, length_x), wcs=wcs_c)
cutout_c_conv = convolve_fft( cutout_c.data, hdu_k.data, allow_huge=True)

cutout_wcs = cutout_c.wcs.to_header()

hdu1 = fits.HDUList()
hdu1.append(fits.PrimaryHDU(header=cutout_wcs))
hdu1.append(fits.ImageHDU())
hdu1[0].data = cutout_c_conv

hdu1.writeto('/Volumes/BEAGLE1/CANDELS_data/convolved_images/%s_f%s_f160w_convolved.fits' % (field, b), overwrite=True)

hdulist_c.close()
hdulist_k.close()

if field == 'goodsn':
    hdulist_160.close()